In [1]:
%matplotlib inline

In [2]:
import time

import gluoncv as gcv
from gluoncv.utils import try_import_cv2
cv2 = try_import_cv2()
import mxnet as mx

In [3]:
gcv.__version__

'0.8.0'

In [4]:
# Load the model
net = gcv.model_zoo.get_model('yolo3_mobilenet1.0_coco', pretrained=True)
# Compile the model for faster speed
net.hybridize()

In [5]:
# Load the webcam handler
cap = cv2.VideoCapture(0)
time.sleep(1) ### letting the camera autofocus

In [6]:
axes = None
NUM_FRAMES = 60 # you can change this
for i in range(NUM_FRAMES):
    # Load frame from the camera
    ret, frame = cap.read()

    if ret:
        # Image pre-processing
        frame = mx.nd.array(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)).astype('uint8')
        rgb_nd, frame = gcv.data.transforms.presets.ssd.transform_test(frame, short=512, max_size=700)

        # Run frame through network
        class_IDs, scores, bounding_boxes = net(rgb_nd)

        # Display the result
        img = gcv.utils.viz.cv_plot_bbox(frame, bounding_boxes[0], scores[0], class_IDs[0], class_names=net.classes)
        gcv.utils.viz.cv_plot_image(img)
        cv2.waitKey(1)
    else:
        break

In [7]:
cap.release()
cv2.destroyAllWindows()